In [ ]:
import httpx
from pathlib import PosixPath

BASE_URL = "https://iceportal.de"
ICE_API = PosixPath("api1")
BAP_API = PosixPath("bap/api")
URL_STATUS = ICE_API / "rs/status"
URL_TRIP = ICE_API / "rs/tripInfo/trip"
URL_BAP_STATUS = BAP_API / "bap-service-status"

client = httpx.Client()
try:
    with httpx.Client(base_url=BASE_URL) as client:
        headers = {'Accept': 'application/json'}
        r_status = client.get(str(URL_STATUS), headers=headers)
        r_trip = client.get(str(URL_TRIP), headers=headers)
        r_bap_status = client.get(str(URL_BAP_STATUS), headers=headers)
finally:
    client.close()

status = r_status.json()
trip = r_trip.json()
bap_status = r_bap_status.json()

In [ ]:
status

In [ ]:
trip

In [ ]:
bap_status

In [ ]:
import folium

m = folium.Map(location=(status["latitude"], status["longitude"]))

folium.Marker(
    location=[status["latitude"], status["longitude"]],
    tooltip=status["tzn"],
    popup=f"{status["trainType"]}\n Geschwindigkeit: {status["speed"]}",
    icon=folium.Icon(color="red"),
).add_to(m)

track_coordinates = list([(stop["station"]["geocoordinates"]["latitude"], stop["station"]["geocoordinates"]["longitude"]) for stop in trip["trip"]["stops"]])

folium.PolyLine(track_coordinates, tooltip="Track").add_to(m)

m